In [1]:
import numpy as np 
import pandas as pd
import os
# import matplotlib.pyplot as plt
# from PIL import Image, ImageDraw, ImageEnhance
from tqdm.notebook import tqdm

# import cv2
import re
import time

import sys
sys.path.append('../')
from retinanet import coco_eval
from retinanet import csv_eval
from retinanet import model
# from retinanet import retina
from retinanet.dataloader import *
from retinanet.anchors import Anchors
from retinanet.losses import *
from retinanet.scheduler import *
from retinanet.parallel import DataParallelModel, DataParallelCriterion

# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder

#Torch
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.optim import Adam, lr_scheduler
import torch.optim as optim


In [2]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print ('Available devices ', torch.cuda.device_count())

# print ('Current cuda device ', torch.cuda.current_device())
# print(torch.cuda.get_device_name(device))

# GPU 할당 변경하기
GPU_NUM = 5 # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device) # change allocation of current GPU
print(device)
print ('Current cuda device ', torch.cuda.current_device()) # check
device_ids = [5,6]
# device_ids = [4,3,2]

cuda:5
Current cuda device  5


In [3]:
# # torch.load(PATH_TO_WEIGHTS, map_location=device)
# %time
# PATH_TO_WEIGHTS = '../coco_resnet_50_map_0_335_state_dict.pt'
# retinanet = model.resnet50(num_classes=2, device=device)
# retinanet.load_state_dict(torch.load(PATH_TO_WEIGHTS, map_location=device), strict=False)


In [4]:
# state_dict = model.state_dict()
# state_dict['classifier.weight'] = torch.randn(10, 10)
# model.load_state_dict(state_dict)

In [5]:
# state_dict = pretrained_retinanet.state_dict()
# state_dict['bn1.bias'] = torch.zeros([64])
# state_dict['bn1.bias']
# pretrained_retinanet.load_state_dict(state_dict)
# pretrained_retinanet.state_dict()['bn1.bias']

In [6]:
%time
PATH_TO_WEIGHTS = '../coco_resnet_50_map_0_335_state_dict.pt'
pre_retinanet = model.resnet50(num_classes=80, device=device)
pre_retinanet.load_state_dict(torch.load(PATH_TO_WEIGHTS, map_location=device), strict=False)
pre_retinanet.classificationModel.output = nn.Conv2d(256, 18, kernel_size=3, padding=1)

retinanet = model.resnet50(num_classes=2, device=device)
retinanet.load_state_dict(pre_retinanet.state_dict(), strict=False)
del pre_retinanet


CPU times: user 3 µs, sys: 4 µs, total: 7 µs
Wall time: 16.2 µs
num_features_in of ResidualAfterFPN : 256
num_features_in of ResidualAfterFPN : 256


In [7]:
state_dict = retinanet.residualafterFPN.state_dict()
# state_dict = retinanet.state_dict()
for s in state_dict:
    if 'bn' in s :
        if 'weight' in s :
            print(s)
            shape = state_dict[s].shape
            state_dict[s] = torch.zeros(shape)
        elif 'bias' in s :
            print(s)
            shape = state_dict[s].shape
#            state_dict[s] = torch.zeros(shape)
            state_dict[s] = torch.ones(shape)            
#         print(residual_state_dict[s])
retinanet.residualafterFPN.load_state_dict(state_dict)

bn1.weight
bn1.bias
bn3.weight
bn3.bias
bn4.weight
bn4.bias


<All keys matched successfully>

In [8]:
# retinanet.to(device)
retinanet = torch.nn.DataParallel(retinanet, device_ids = [5,6], output_device=GPU_NUM).to(device)
# retinanet = DataParallelModel(retinanet, device_ids = device_ids)
#retinanet.to(device)
# retinanet.cuda()
# retinanet.module.freeze_ex_bn()
#retinanet.module.freeze_ex_bn(False)
#retinanet.freeze_ex_bn(False)

In [9]:
# for n, p in retinanet.named_parameters():
#     #print(n)
#     if 'bn' not in n :
#         if 'fpn' not in n and 'residualafterFPN' not in n and 'regressionModel' not in n and 'classificationModel' not in n :
#             #print(param)
#             print(n)
#             p.requires_grad = False
    

In [10]:
# for k, p in zip(retinanet.module.state_dict(), retinanet.module.parameters()) :
#     if 'bn' not in k :
# #         print(k)
#         p.requires_grad = False
# #         print(p)
                

In [11]:
# train_info = np.load('../data/train.npy', allow_pickle=True, encoding='latin1').item()
# train_info

batch_size = 32
dataset_train = PapsDataset('../data/', set_name='train_2class',
                            transform=train_transforms)

train_data_loader = DataLoader(
    dataset_train,
    batch_size=batch_size,
    shuffle=True,
    num_workers=16,
    pin_memory=True,
    prefetch_factor=1,
    collate_fn=collate_fn
)

loading annotations into memory...
Done (t=0.89s)
creating index...
index created!


In [12]:
criterion = FocalLoss(device)
criterion = criterion.to(device)
retinanet.training = True

# https://gaussian37.github.io/dl-pytorch-lr_scheduler/
optimizer = optim.Adam(retinanet.parameters(), lr = 1e-8)
scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=20, T_mult=2, eta_max=0.0008,  T_up=5, gamma=0.5)
# CosineAnnealingWarmRestarts

In [13]:
# state_dict = retinanet.module.residualafterFPN.state_dict()
# for s in state_dict:
#     if 'bn' in s :
#         if 'weight' in s :
#             print(s)
#             print(state_dict[s])
#         elif 'bias' in s :
#             print(s)
#             print(state_dict[s])

In [14]:
retinanet.module.freeze_ex_bn(False)

In [ ]:
#for i, data in enumerate(tqdm(train_data_loader)) :
EPOCH_NUM = 160
loss_per_epoch = 0.6
optimizer.param_groups[0]["lr"] = 0.00002
for epoch in range(EPOCH_NUM) :
    if epoch == int(EPOCH_NUM *0.2) :
        retinanet.module.freeze_ex_bn(True)
    total_loss = 0
    tk0 = tqdm(train_data_loader, total=len(train_data_loader), leave=False)
    EPOCH_LEARING_RATE = optimizer.param_groups[0]["lr"]
    start_time = time.time()
#     print("*****{}th epoch, learning rate {}".format(epoch, EPOCH_LEARING_RATE))

    for step, data in enumerate(tk0) :
        if step > len(train_data_loader)/4 and epoch < int(EPOCH_NUM*0.8)  :
            break
        images, box, label, targets = data
        batch_size = len(images)

        c, h, w = images[0].shape
        images = torch.cat(images).view(-1, c, h, w).to(device)
        targets = [ t.to(device) for t in targets]

        outputs = retinanet([images, targets])
        classification, regression, anchors, annotations = (outputs)
        classification_loss, regression_loss = criterion(classification, regression, anchors, annotations)

        classification_loss = classification_loss.mean()
        regression_loss = regression_loss.mean()
        loss = classification_loss + regression_loss 
        total_loss += loss.item()
        
        if step % 5 == 0:
            tk0.set_postfix(lr=optimizer.param_groups[0]["lr"], batch_loss=loss.item(), cls_loss=classification_loss.item(), 
                            reg_loss=regression_loss.item(), avg_loss=total_loss/(step+1))

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(retinanet.parameters(), 0.02)
        optimizer.step()   

    print('{}th epochs loss is {}'.format(epoch, total_loss/(step+1)))
    if loss_per_epoch > total_loss/(step+1):
        print('best model is saved')
        torch.save(retinanet.state_dict(), '../trained_models/resnet50_320_ex_bn/best_model.pt')
        loss_per_epoch = total_loss/(step+1)
        
    scheduler.step()
#     print('epoch training time is ', time.time() - start_time)

torch.save(retinanet.state_dict(), '../trained_models/resnet50_320_ex_bn/model.pt')


  0%|          | 0/465 [00:00<?, ?it/s]

*****0th epoch, learning rate 2e-05
0th epochs loss is 113.3125921336271
epoch training time is  227.86394429206848


  0%|          | 0/465 [00:00<?, ?it/s]

*****1th epoch, learning rate 0.000160008
1th epochs loss is 0.915842695003849
epoch training time is  228.23399019241333


  0%|          | 0/465 [00:00<?, ?it/s]

*****2th epoch, learning rate 0.000320006
2th epochs loss is 0.5899777324017832
best model is saved
epoch training time is  142.6881980895996


  0%|          | 0/465 [00:00<?, ?it/s]

*****3th epoch, learning rate 0.00048000399999999997
3th epochs loss is 0.6253384268384868
epoch training time is  199.03216314315796


  0%|          | 0/465 [00:00<?, ?it/s]

*****4th epoch, learning rate 0.0006400020000000001


In [ ]:
# for n, p in retinanet.named_parameters():
#     #print(n)
#     if 'bn' not in n :
#         if 'fpn' not in n and 'residualafterFPN' not in n and 'regressionModel' not in n and 'classificationModel' not in n :
#             #print(param)
#             print(n)
#             print(p[0])
#             #p.requires_grad = False

In [ ]:
# for n, p in retinanet.named_parameters():
#     #print(n)
#     if 'bn' not in n :
#         if 'fpn' not in n and 'residualafterFPN' not in n and 'regressionModel' not in n and 'classificationModel' not in n :
#             #print(param)
#             print(n)
#             print(p[0])
#             #p.requires_grad = False
    

In [ ]:
# state_dict = retinanet.residualafterFPN.state_dict()
# for s in state_dict:
#     if 'bn' in s :
#         if 'weight' in s :
#             print(s)
#             print(state_dict[s])
#         elif 'bias' in s :
#             print(s)
#             print(state_dict[s])

In [ ]:
dataset_val = PapsDataset('../data/', set_name='val_2class',
                            transform=val_transforms)

val_data_loader = DataLoader(
    dataset_val,
    batch_size=2,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

In [ ]:
# retinanet.load_state_dict(torch.load('../trained_models/resnet50_320_ex_bn/model.pt'))

In [ ]:
from pycocotools.cocoeval import COCOeval
import json
import torch

In [ ]:
retinanet.eval()
start_time = time.time()
threshold = 0.1
results = []
GT_results = []
image_ids = []
cnt = 0
scores_list = []

for index, data in enumerate(tqdm(val_data_loader)) :
    if cnt > 50 :
        break
    cnt += 1
    with torch.no_grad():        
        images, tbox, tlabel, targets = data
        batch_size = len(images)
#         print(tbox)
#         print(len(tbox[0]))

        c, h, w = images[0].shape
        images = torch.cat(images).view(-1, c, h, w).to(device)

        outputs = retinanet(images)
        scores, labels, boxes = (outputs)
        
        scores = scores.cpu()
        labels = labels.cpu()
        boxes  = boxes.cpu()  
        
        scores_list.append(scores)

        if boxes.shape[0] > 0:
            # change to (x, y, w, h) (MS COCO standard)
            boxes[:, 2] -= boxes[:, 0]
            boxes[:, 3] -= boxes[:, 1]
#             print(boxes)

            # compute predicted labels and scores
            #for box, score, label in zip(boxes[0], scores[0], labels[0]):
            for box_id in range(boxes.shape[0]):
                score = float(scores[box_id])
                label = int(labels[box_id])
                box = boxes[box_id, :]

                # scores are sorted, so we can break
                if score < threshold:
                    break

                # append detection for each positively labeled class
                image_result = {
                    'image_id'    : dataset_val.image_ids[index],
                    'category_id' : dataset_val.label_to_coco_label(label),
                    'score'       : float(score),
                    'bbox'        : box.tolist(),
                }

                # append detection to results
                results.append(image_result)
                
        if len(tbox[0]) > 0:    

            # compute predicted labels and scores
            #for box, score, label in zip(boxes[0], scores[0], labels[0]):
            for box_id in range(len(tbox[0])):
                score = float(0.99)
                label = (tlabel[0][box_id])
                box = list(tbox[0][box_id])
                box[2] -= box[0]
                box[3] -= box[1]             

                # append detection for each positively labeled class
                image_result = {
                    'image_id'    : dataset_val.image_ids[index],
                    'category_id' : dataset_val.label_to_coco_label(label),
                    'score'       : float(score),
                    'bbox'        : list(box),
                }

                # append detection to results
                GT_results.append(image_result)                

        # append image to list of processed images
        image_ids.append(dataset_val.image_ids[index])

        # print progress
        print('{}/{}'.format(index, len(dataset_val)), end='\r')    

if not len(results):
    print('No object detected')
print('GT_results', len(GT_results))    
print('pred_results', len(results))    

# write output
json.dump(results, open('../trained_models/resnet50_320_ex_bn/{}_bbox_results.json'.format(dataset_val.set_name), 'w'), indent=4)
# write GT
json.dump(GT_results, open('../trained_models/resnet50_320_ex_bn/{}_GTbbox_results.json'.format(dataset_val.set_name), 'w'), indent=4)     

print('validation time :', time.time() - start_time)



In [ ]:
# load results in COCO evaluation tool
coco_true = dataset_val.coco
coco_pred = coco_true.loadRes('../trained_models/resnet50_320_ex_bn/{}_bbox_results.json'.format(dataset_val.set_name))
coco_gt = coco_true.loadRes('../trained_models/resnet50_320_ex_bn/{}_GTbbox_results.json'.format(dataset_val.set_name))

# run COCO evaluation
# coco_eval = COCOeval(coco_true, coco_pred, 'bbox')
coco_eval = COCOeval(coco_gt, coco_pred, 'bbox')
coco_eval.params.imgIds = image_ids
# coco_eval.params.catIds = [0]
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()   

In [ ]:
coco_eval.params.catIds

In [ ]:
coco_eval = COCOeval(coco_gt, coco_pred, 'bbox')
coco_eval.params.imgIds = image_ids
coco_eval.params.catIds = [0]
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()   

In [ ]:
coco_eval = COCOeval(coco_gt, coco_pred, 'bbox')
coco_eval.params.imgIds = image_ids
coco_eval.params.catIds = [1]
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()   